In [1]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import json

sess = sagemaker.Session()

role = get_execution_role()
print(
    role
)  # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = "demosagemakerbucket0" # Replace with your own bucket name if needed
print(bucket)
prefix = "demosagemakerbucket0/fasttext/pretrained"

arn:aws:iam::906756132937:role/service-role/AmazonSageMaker-ExecutionRole-20210726T113143
demosagemakerbucket0


In [2]:
region_name = boto3.Session().region_name

In [3]:
container = sagemaker.amazon.amazon_estimator.image_uris.retrieve("blazingtext", region_name, "1")
print("Using SageMaker BlazingText container: {} ({})".format(container, region_name))

Using SageMaker BlazingText container: 501404015308.dkr.ecr.ap-northeast-1.amazonaws.com/blazingtext:1 (ap-northeast-1)


In [5]:
!wget -O model.bin https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin


--2021-07-29 05:03:22--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Saving to: ‘model.bin’

model.bin           100%[===================>] 125.18M  13.8MB/s    in 10s     

2021-07-29 05:03:32 (12.6 MB/s) - ‘model.bin’ saved [131266198/131266198]



In [6]:
!tar -czvf langid.tar.gz model.bin
model_location = sess.upload_data("langid.tar.gz", bucket=bucket, key_prefix=prefix)
print(model_location)
!rm langid.tar.gz model.bin

model.bin
s3://demosagemakerbucket0/demosagemakerbucket0/fasttext/pretrained/langid.tar.gz


In [7]:

lang_id = sagemaker.Model(
    image_uri=container, model_data=model_location, role=role, sagemaker_session=sess
)
#lang_id.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge",accelerator_type='ml.eia2.xlarge')
lang_id.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer

predictor = sagemaker.Predictor(
    endpoint_name=lang_id.endpoint_name,
    sagemaker_session=sess,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

--------------!

In [8]:
sentences = [
    "hi which language is this?",
    "mon nom est Pierre",
    "Dem Jungen gab ich einen Ball.",
    "আমি বাড়ি যাবো.",
]
payload = {"instances": sentences}

In [9]:
payload

{'instances': ['hi which language is this?',
  'mon nom est Pierre',
  'Dem Jungen gab ich einen Ball.',
  'আমি বাড়ি যাবো.']}

In [16]:
payload_test= {
  "instances": [

    'I eat rice' ,    
    'I go'     
  ]
}

In [10]:
predictions = predictor.predict(payload_test)
print(predictions)

NameError: name 'payload_test' is not defined

In [11]:
predictions = predictor.predict(payload)
print(predictions)

[{'label': ['__label__en'], 'prob': [0.9948582053184509]}, {'label': ['__label__fr'], 'prob': [0.9984669089317322]}, {'label': ['__label__de'], 'prob': [0.9946573972702026]}, {'label': ['__label__bn'], 'prob': [0.9997219443321228]}]


In [13]:
import copy

predictions_copy = copy.deepcopy(
    predictions
)  # Copying predictions object because we want to change the labels in-place
for output in predictions_copy:
    output["label"] = output["label"][0][9:].upper()  # __label__ has length of 9

print(predictions_copy)


[{'label': 'EN', 'prob': [0.9948582053184509]}, {'label': 'FR', 'prob': [0.9984669089317322]}, {'label': 'DE', 'prob': [0.9946573972702026]}, {'label': 'BN', 'prob': [0.9997219443321228]}]


In [14]:
predictor.delete_endpoint()

In [3]:
!ls ..


anaconda3  Nvidia_Cloud_EULA.pdf  sample-notebooks	       tools
examples   README		  sample-notebooks-1627625996  tutorials
LICENSE    SageMaker		  src


In [4]:
!pwd

/home/ec2-user/SageMaker


In [7]:
!ls

caltech-256-60-train.rec  object2vec_movie_recommendation_2021-07-28
caltech-256-60-val.rec	  object2vec_movie_recommendation_2021-07-29
lost+found		  Untitled1.ipynb
NLP.ipynb		  Untitled.ipynb


In [8]:
!rm -rf *.rec

In [9]:
!rm -rf object2vec_movie_recommendation_*

In [10]:
!ls

lost+found  NLP.ipynb  SageMakerWorks  Untitled1.ipynb	Untitled.ipynb


In [13]:
!ls SageMakerWorks/


NLP.ipynb  Untitled1.ipynb  Untitled.ipynb


In [12]:
!cp *.ipynb SageMakerWorks/

In [14]:
cd SageMakerWorks/


/home/ec2-user/SageMaker/SageMakerWorks


In [15]:
!ls

NLP.ipynb  Untitled1.ipynb  Untitled.ipynb


In [16]:
!git add .

In [17]:
!git commit -m "added sagemaker works"


[master (root-commit) 0838855] added sagemaker works
 Committer: EC2 Default User <ec2-user@ip-172-16-2-89.ap-northeast-1.compute.internal>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 3 files changed, 915 insertions(+)
 create mode 100644 NLP.ipynb
 create mode 100644 Untitled.ipynb
 create mode 100644 Untitled1.ipynb


In [18]:
!git config --global user.name "Ahmed Mustahid"
!git config --global user.email einsteinium25@gmail.com

In [19]:
!git commit --amend --reset-author


hint: Waiting for your editor to close the file... 7="~/SageMaker/SageMakerWorks/.git/COMMIT_EDITMSG" 14L, 312Cadded sagemaker works

# Please enter the commit message for your changes. Lines starting
# with '#' will be ignored, and an empty message aborts the commit.
#
# On branch master
#
# Initial commit
#
# Changes to be committed:
#new file:   NLP.ipynb
#new file:   Untitled.ipynb
#new file:   Untitled1.ipynb
#
~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                 

In [20]:
!git status

On branch master
Your branch is based on 'origin/master', but the upstream is gone.
  (use "git branch --unset-upstream" to fixup)

nothing to commit, working tree clean


In [21]:
!git push origin main

error: src refspec main does not match any.
error: failed to push some refs to 'https://github.com/ahmedmustahid/SageMakerWorks.git'


In [22]:
!ls


NLP.ipynb  Untitled1.ipynb  Untitled.ipynb


In [2]:
! cd .. && rm -rf SageMakerWorks/